#  Whole Genome Shotgun metagenomics: Taxonomic Binning Pipeline

We have two fastqc files to process:
1. *Microbiome1_200k* that contains $200,000$ paired end reads from total DNA extracted from human saliva. So this is a microbiome.


2. *Vir1_100k* that contains $100,000$ paired end reads from the same saliva sample but after purification of viral particles. So this is a virome.


In [105]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd

## Check files
Check the number of reads of our files and look at the header with grep and head commands

In [152]:
%%bash
ssh microbioinf@192.168.56.101 2>/dev/null /bin/bash <<'EOT'
cd Documentos/Tema_4a
head -n 1 *fq
grep -c "@M02255" *fq
EOT

==> Bact1_R1_200000.fq <==
@M02255:131:000000000-AJC6R:1:2102:25217:13392_1:N:0:ACAGTG

==> Bact1_R2_200000.fq <==
@M02255:131:000000000-AJC6R:1:2102:25217:13392_2:N:0:ACAGTG

==> Vir1_R1_100000.fq <==
@M02255:131:000000000-AJC6R:1:1105:23249:10170_1:N:0:AGTCAA

==> Vir1_R2_100000.fq <==
@M02255:131:000000000-AJC6R:1:1105:23249:10170_2:N:0:AGTCAA
Bact1_R1_200000.fq:200000
Bact1_R2_200000.fq:200000
Vir1_R1_100000.fq:100000
Vir1_R2_100000.fq:100000


## Quality (fastqc) ##

In [153]:
%%bash
ssh microbioinf@192.168.56.101 2>/dev/null /bin/bash <<'EOT'
export PATH=$PATH:/home/microbioinf/miniconda3/bin
cd Documentos/Tema_4a
fastqc Vir1_R1_100000.fq -o fastQC_output/
fastqc Vir1_R2_100000.fq -o fastQC_output/
fastqc Bact1_R1_200000.fq -o fastQC_output/
fastqc Bact1_R2_200000.fq -o fastQC_output/
EOT

Analysis complete for Vir1_R1_100000.fq
Analysis complete for Vir1_R2_100000.fq
Analysis complete for Bact1_R1_200000.fq
Analysis complete for Bact1_R2_200000.fq


## Trimming and decontaminating
Trimming poor quality ends and short sequences (**Trimmomatic**) and removal of reads aligning to the human and phiX174 genomes (***bowtie2**). The later one is a contaminant used as spike by Illumina kits to control quality of the sequencing process.

We are only filtering only R1 files because forward reads have usually better quality than reverse reads. 

### Processing

In [92]:
%%bash
ssh microbioinf@192.168.56.101 2>/dev/null /bin/bash <<'EOT'
export PATH=$PATH:/home/microbioinf/miniconda3/bin
cd Documentos/Tema_4a
kneaddata -i Bact1_R1_200000.fq -o kneaddata_out \
-db /home/shared/bowtiedb/GRCh38_PhiX \
--trimmomatic /home/microbioinf/miniconda3/pkgs/trimmomatic-0.38-1/share/trimmomatic-0.38-1/ \
-t 2 --trimmomatic-options "SLIDINGWINDOW:4:20 MINLEN:200" \
--bowtie2-options " --sensitive" --remove-intermediate-output
EOT

Initial number of reads ( /home/microbioinf/Documentos/Tema_4a/Bact1_R1_200000.fq ): 200000
Running Trimmomatic ... 
Total reads after trimming ( /home/microbioinf/Documentos/Tema_4a/kneaddata_out/Bact1_R1_200000_kneaddata.trimmed.fastq ): 127652
Decontaminating ...
Running bowtie2 ... 
Total reads after removing those found in reference database ( /home/microbioinf/Documentos/Tema_4a/kneaddata_out/Bact1_R1_200000_kneaddata_GRCh38_PhiX_bowtie2_clean.fastq ): 32358
Total reads after merging results from multiple databases ( /home/microbioinf/Documentos/Tema_4a/kneaddata_out/Bact1_R1_200000_kneaddata.fastq ): 32358

Final output file created: 
/home/microbioinf/Documentos/Tema_4a/kneaddata_out/Bact1_R1_200000_kneaddata.fastq



In [91]:
%%bash
ssh microbioinf@192.168.56.101 2>/dev/null /bin/bash <<'EOT'
export PATH=$PATH:/home/microbioinf/miniconda3/bin
cd Documentos/Tema_4a
kneaddata -i Vir1_R1_100000.fq -o kneaddata_out \
-db /home/shared/bowtiedb/GRCh38_PhiX \
--trimmomatic /home/microbioinf/miniconda3/pkgs/trimmomatic-0.38-1/share/trimmomatic-0.38-1/ \
-t 2 --trimmomatic-options "SLIDINGWINDOW:4:20 MINLEN:200" \
--bowtie2-options " --sensitive" --remove-intermediate-output
EOT

Initial number of reads ( /home/microbioinf/Documentos/Tema_4a/Vir1_R1_100000.fq ): 100000
Running Trimmomatic ... 
Total reads after trimming ( /home/microbioinf/Documentos/Tema_4a/kneaddata_out/Vir1_R1_100000_kneaddata.trimmed.fastq ): 63602
Decontaminating ...
Running bowtie2 ... 
Total reads after removing those found in reference database ( /home/microbioinf/Documentos/Tema_4a/kneaddata_out/Vir1_R1_100000_kneaddata_GRCh38_PhiX_bowtie2_clean.fastq ): 63331
Total reads after merging results from multiple databases ( /home/microbioinf/Documentos/Tema_4a/kneaddata_out/Vir1_R1_100000_kneaddata.fastq ): 63331

Final output file created: 
/home/microbioinf/Documentos/Tema_4a/kneaddata_out/Vir1_R1_100000_kneaddata.fastq



### Process statistics

In [96]:
%%bash
ssh microbioinf@192.168.56.101 2>/dev/null /bin/bash <<'EOT'
export PATH=$PATH:/home/microbioinf/miniconda3/bin
cd Documentos/Tema_4a/kneaddata_out
kneaddata_read_count_table --input ./ --output kneaddata_read_counts.txt
EOT

Reading log: ./Bact1_R1_200000_kneaddata.log
Reading log: ./Vir1_R1_100000_kneaddata.log
Read count table written: kneaddata_read_counts.txt


In [154]:
data = """
cat Documentos/Tema_4a/kneaddata_out/kneaddata_read_counts.txt
EOT
"""
output = !ssh microbioinf@192.168.56.101 2>/dev/null /bin/bash <<'EOT' {data}

data = []
# To list of lists
for row in output:
    data.append(row.split('\t'))
# To dataframe
df = pd.DataFrame(data[1:], columns=data[0])
df.style.hide_index().set_properties(**{'text-align': 'right', 'font-family' : 'courier', 'color' : 'darkgreen', "font-size" : "11pt"}).\
set_properties(**{'text-align': 'right', 'font-family' : 'courier', 'color' : 'darkblue', "font-size" : "12pt"}, subset=['Sample'])

Sample,raw single,trimmed single,decontaminated GRCh38_PhiX single,final single
Bact1_R1_200000_kneaddata,200000,127652,32358,32358
Vir1_R1_100000_kneaddata,100000,63602,63331,63331


With grep we can identify the non-contaminated high-quality files

In [155]:
%%bash
ssh microbioinf@192.168.56.101 2>/dev/null /bin/bash <<'EOT'
cd Documentos/Tema_4a/kneaddata_out
grep -c "@M02255:" *fastq
EOT

Bact1_R1_200000_kneaddata.fastq:32358
Bact1_R1_200000_kneaddata_GRCh38_PhiX_bowtie2_contam.fastq:95294
Vir1_R1_100000_kneaddata.fastq:63331
Vir1_R1_100000_kneaddata_GRCh38_PhiX_bowtie2_contam.fastq:271


### Check quality

In [156]:
%%bash
ssh microbioinf@192.168.56.101 2>/dev/null /bin/bash <<'EOT'
cd Documentos/Tema_4a/kneaddata_out
mkdir fastQC_HighQuality
export PATH=$PATH:/home/microbioinf/miniconda3/bin
fastqc Bact1_R1_200000_kneaddata.fastq -o fastQC_HighQuality/
fastqc Vir1_R1_100000_kneaddata.fastq -o fastQC_HighQuality/
EOT

Analysis complete for Bact1_R1_200000_kneaddata.fastq
Analysis complete for Vir1_R1_100000_kneaddata.fastq


## Blastx comparison with a viral protein database

 We are going to use a Refseq database of viral proteins (around 100Mb) from ncbi (ftp://ftp.ncbi.nlm.nih.gov/refseq/release/viral/), and you have to download it in two separated files that can be joined into one with cat.
 

### Set up the reference database for diamond

This will create a binary DIAMOND database file with the name: *viralprotein.dmnd*

In [157]:
%%bash
ssh microbioinf@192.168.56.101 2>/dev/null /bin/bash <<'EOT'
cd Documentos/Tema_4a
export PATH=$PATH:/home/microbioinf/miniconda3/bin
diamond makedb --in viral.protein.faa -d viralprotein
EOT

diamond v0.8.36.98 | by Benjamin Buchfink <buchfink@gmail.com>
Check http://github.com/bbuchfink/diamond for updates.

#CPU threads: 3
Database file: viral.protein.faa
Opening the database file...  [0.015813s]
Loading sequence data (0 sequences processed)...  [0.149662s]
Loading sequence data (100000 sequences processed)...  [0.142183s]
Loading sequence data (200000 sequences processed)...  [0.158145s]
Loading sequence data (300000 sequences processed)...  [0.040807s]
Writing trailer...  [0.003161s]
Closing the input file...  [4e-05s]
Closing the database file...  [0.061918s]
Processed 323029 sequences, 82978170 letters.
Total time = 0.572598s
